In [1]:
import json

with open("data/corona.json") as f:
    data = json.loads(f.read())


TRAIN_DATA = []

for (text, annot) in data:
    new_anno = []
    for anno in annot["entities"]:
        st, end, label = anno
        new_anno.append((st, end, label))
    TRAIN_DATA.append((text, {"entities": new_anno}))


labels = ['Pathogen', 'MedicalCondition', 'Medicine']

In [2]:
import random
import spacy
from spacy.training import Example
spacy.prefer_gpu()

True

In [3]:
nlp = spacy.blank("en")
ner = nlp.add_pipe("ner") 

print(ner)
print(nlp.meta)

for ent in labels:
   ner.add_label(ent)

print(ner.labels)

epochs = 25

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

{'lang': 'en', 'name': 'pipeline', 'version': '0.0.0', 'spacy_version': '>=3.4.3,<3.5.0', 'description': '', 'author': '', 'email': '', 'url': '', 'license': '', 'spacy_git_version': '63673a792', 'vectors': {'width': 0, 'vectors': 0, 'keys': 0, 'name': None, 'mode': 'default'}, 'labels': {'ner': []}, 'pipeline': ['ner'], 'components': ['ner'], 'disabled': []}
('MedicalCondition', 'Medicine', 'Pathogen')


In [4]:
#spacy.require_gpu()
spacy.prefer_gpu()
with nlp.disable_pipes(*other_pipes):

  optimizer = nlp.begin_training()

  for i in range(100):
    random.shuffle(TRAIN_DATA)
    for text, annotation in TRAIN_DATA:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotation)
        nlp.update([example], sgd=optimizer)

doc = nlp("One of the bacterial diseases with the highest disease burden is tuberculosis, caused by Mycobacterium tuberculosis bacteria, which kills about 2 million people a year.")
doc2 = nlp("Pathogenic bacteria contribute to other globally important diseases, such as pneumonia, which can be caused by bacteria such as Streptococcus and Pseudomonas, and foodborne illnesses, which can be caused by bacteria such as Shigella, Campylobacter, and Salmonella. Pathogenic bacteria also cause infections such as tetanus, typhoid fever, diphtheria, syphilis, and leprosy. Pathogenic bacteria are also the cause of high infant mortality rates in developing countries.")

print(doc.ents)
print(doc)

C:\Users\rob\anaconda3\envs\spacy-GPU-env\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Buprenorphine has been shown experimentally (1982â..." with entities "[(0, 13, 'Medicine'), (88, 109, 'MedicalCondition'...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\Users\rob\anaconda3\envs\spacy-GPU-env\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Bats are the most common source of rabies in human..." with entities "[(35, 41, 'MedicalCondition'), (162, 168, 'Medical...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\Users\rob\anaconda3\envs\spacy-GPU-env\lib\site-packages\spacy\training\iob_u

(tuberculosis, Mycobacterium tuberculosis)
One of the bacterial diseases with the highest disease burden is tuberculosis, caused by Mycobacterium tuberculosis bacteria, which kills about 2 million people a year.


In [6]:
spacy.require_cpu()

True